# Úloha č. 2: Test hypotézy

Overte nasledujúcu hypotézu s 95% istotou:

*Pokiaľ bol vinník nehody pod silným vplyvom alkoholu, došlo častejšie k ťažkým zdravotným následkom.*

In [1]:
###
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Načítanie a vyčistenie dát

In [2]:
# Načítanie dát
df = pd.read_pickle("accidents.pkl.gz")

# Vytvorenie DataFrame iba so stĺpcami, ktoré potrebujem
data = df[["p11", "p13a", "p13b"]]

### Štruktúra stĺpcov *p11*, *p13a* a *p13b*

#### *p11*
Predstavuje prítomnosť alkoholu u vinníka nehody. Povolené sú celé čísla z intervalu **<0,9>**.

| Hodnota | Alkohol |
|---|---|
| 0 | nezisťované |
| 1 | áno, < 0.24 ‰ |
| 2 | nie |
| 3 | áno, >= 0.24 & < 0.5 ‰ |
| 4 | pod vplyvom drog |
| 5 | pod vplyvom alkoholu a drog |
| 6 | áno, >= 0.5 & < 0.8 ‰ |
| 7 | áno, >= 0.8 & < 1.0 ‰ |
| 8 | áno, >= 1.0 & < 1.5 ‰ |
| 9 | áno, >= 1.5 ‰ |

Za silný vplyv alkoholu budeme považovať iba ak je **>= 1.5 ‰**.(podľa https://cs.wikipedia.org/wiki/Opilost)


#### *p13a*
Predstavuje počet osôb, ktoré boli usmrtené pri nehode.
Hodnota musí byť celé číslo **>= 0**.

#### *p13b*
Predstavuje počet osôb, ktoré boli ťažko zranené pri nehode.
Hodnota musí byť celé číslo **>= 0**.

### Odstrániť hodnôt mimo rozsah zo stĺpcov *p11*, *p13a* a *p13b*

In [3]:
data = data.query('p11 >= 0 & p11 <= 9 & p13a >= 0 & p13b >= 0')

## Hľadanie regresného modelu
Zistiť, či neexistuje korelácia medzi nezávislými premennými

In [4]:
sns.pairplot(data.sample(1000)[["p11", "p13a","p13b"]])